In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

2022-12-05 14:24:28.950451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 14:24:29.047918: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-05 14:24:29.047934: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-05 14:24:29.063996: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-05 14:24:29.499443: W tensorflow/stream_executor/platform/de

In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-12-05 14:24:30.379850: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (100)
2022-12-05 14:24:30.379891: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (GG): /proc/driver/nvidia/version does not exist
2022-12-05 14:24:30.380086: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 7)                 119       
                                                                 
Total params: 16,663
Trainable params: 16,663
Non-trainable params: 0
____________________________________________________

In [8]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)



Epoch 1/550
1/1 [==============================] - 0s 311ms/step - loss: 1.9443 - accuracy: 0.1429
Epoch 2/550
1/1 [==============================] - 0s 4ms/step - loss: 1.9428 - accuracy: 0.2143
Epoch 3/550
1/1 [==============================] - 0s 3ms/step - loss: 1.9415 - accuracy: 0.1786
Epoch 4/550
1/1 [==============================] - 0s 4ms/step - loss: 1.9404 - accuracy: 0.2500
Epoch 5/550
1/1 [==============================] - 0s 4ms/step - loss: 1.9393 - accuracy: 0.2500
Epoch 6/550
1/1 [==============================] - 0s 4ms/step - loss: 1.9381 - accuracy: 0.2500
Epoch 7/550
1/1 [==============================] - 0s 4ms/step - loss: 1.9369 - accuracy: 0.2500
Epoch 8/550
1/1 [==============================] - 0s 4ms/step - loss: 1.9356 - accuracy: 0.2500
Epoch 9/550
1/1 [==============================] - 0s 4ms/step - loss: 1.9344 - accuracy: 0.2500
Epoch 10/550
1/1 [==============================] - 0s 4ms/step - loss: 1.9332 - accuracy: 0.2500
Epoch 11/550
1/1 [=========

1/1 [==============================] - 0s 4ms/step - loss: 1.7815 - accuracy: 0.2500
Epoch 85/550
1/1 [==============================] - 0s 3ms/step - loss: 1.7777 - accuracy: 0.2500
Epoch 86/550
1/1 [==============================] - 0s 3ms/step - loss: 1.7738 - accuracy: 0.2857
Epoch 87/550
1/1 [==============================] - 0s 4ms/step - loss: 1.7697 - accuracy: 0.2857
Epoch 88/550
1/1 [==============================] - 0s 3ms/step - loss: 1.7656 - accuracy: 0.2857
Epoch 89/550
1/1 [==============================] - 0s 4ms/step - loss: 1.7613 - accuracy: 0.2857
Epoch 90/550
1/1 [==============================] - 0s 3ms/step - loss: 1.7569 - accuracy: 0.3214
Epoch 91/550
1/1 [==============================] - 0s 4ms/step - loss: 1.7524 - accuracy: 0.4286
Epoch 92/550
1/1 [==============================] - 0s 4ms/step - loss: 1.7477 - accuracy: 0.4286
Epoch 93/550
1/1 [==============================] - 0s 4ms/step - loss: 1.7430 - accuracy: 0.4286
Epoch 94/550
1/1 [===============

1/1 [==============================] - 0s 3ms/step - loss: 1.2671 - accuracy: 0.4286
Epoch 168/550
1/1 [==============================] - 0s 3ms/step - loss: 1.2623 - accuracy: 0.4286
Epoch 169/550
1/1 [==============================] - 0s 3ms/step - loss: 1.2577 - accuracy: 0.4286
Epoch 170/550
1/1 [==============================] - 0s 3ms/step - loss: 1.2532 - accuracy: 0.4286
Epoch 171/550
1/1 [==============================] - 0s 4ms/step - loss: 1.2486 - accuracy: 0.4286
Epoch 172/550
1/1 [==============================] - 0s 3ms/step - loss: 1.2443 - accuracy: 0.4286
Epoch 173/550
1/1 [==============================] - 0s 3ms/step - loss: 1.2398 - accuracy: 0.4286
Epoch 174/550
1/1 [==============================] - 0s 3ms/step - loss: 1.2356 - accuracy: 0.4286
Epoch 175/550
1/1 [==============================] - 0s 4ms/step - loss: 1.2313 - accuracy: 0.4286
Epoch 176/550
1/1 [==============================] - 0s 4ms/step - loss: 1.2271 - accuracy: 0.4286
Epoch 177/550
1/1 [=====

1/1 [==============================] - 0s 3ms/step - loss: 0.9640 - accuracy: 0.6071
Epoch 250/550
1/1 [==============================] - 0s 3ms/step - loss: 0.9598 - accuracy: 0.6071
Epoch 251/550
1/1 [==============================] - 0s 3ms/step - loss: 0.9557 - accuracy: 0.6071
Epoch 252/550
1/1 [==============================] - 0s 3ms/step - loss: 0.9515 - accuracy: 0.6071
Epoch 253/550
1/1 [==============================] - 0s 4ms/step - loss: 0.9472 - accuracy: 0.6071
Epoch 254/550
1/1 [==============================] - 0s 4ms/step - loss: 0.9429 - accuracy: 0.6071
Epoch 255/550
1/1 [==============================] - 0s 3ms/step - loss: 0.9386 - accuracy: 0.6429
Epoch 256/550
1/1 [==============================] - 0s 4ms/step - loss: 0.9342 - accuracy: 0.6429
Epoch 257/550
1/1 [==============================] - 0s 4ms/step - loss: 0.9298 - accuracy: 0.6429
Epoch 258/550
1/1 [==============================] - 0s 4ms/step - loss: 0.9253 - accuracy: 0.6429
Epoch 259/550
1/1 [=====

1/1 [==============================] - 0s 4ms/step - loss: 0.5288 - accuracy: 0.8929
Epoch 332/550
1/1 [==============================] - 0s 4ms/step - loss: 0.5236 - accuracy: 0.8929
Epoch 333/550
1/1 [==============================] - 0s 4ms/step - loss: 0.5183 - accuracy: 0.8929
Epoch 334/550
1/1 [==============================] - 0s 4ms/step - loss: 0.5131 - accuracy: 0.8929
Epoch 335/550
1/1 [==============================] - 0s 4ms/step - loss: 0.5080 - accuracy: 0.8929
Epoch 336/550
1/1 [==============================] - 0s 4ms/step - loss: 0.5029 - accuracy: 0.8929
Epoch 337/550
1/1 [==============================] - 0s 4ms/step - loss: 0.4977 - accuracy: 0.8929
Epoch 338/550
1/1 [==============================] - 0s 4ms/step - loss: 0.4927 - accuracy: 0.8929
Epoch 339/550
1/1 [==============================] - 0s 3ms/step - loss: 0.4876 - accuracy: 0.8929
Epoch 340/550
1/1 [==============================] - 0s 4ms/step - loss: 0.4826 - accuracy: 0.8929
Epoch 341/550
1/1 [=====

1/1 [==============================] - 0s 3ms/step - loss: 0.1977 - accuracy: 1.0000
Epoch 414/550
1/1 [==============================] - 0s 3ms/step - loss: 0.1951 - accuracy: 1.0000
Epoch 415/550
1/1 [==============================] - 0s 4ms/step - loss: 0.1924 - accuracy: 1.0000
Epoch 416/550
1/1 [==============================] - 0s 4ms/step - loss: 0.1899 - accuracy: 1.0000
Epoch 417/550
1/1 [==============================] - 0s 3ms/step - loss: 0.1873 - accuracy: 1.0000
Epoch 418/550
1/1 [==============================] - 0s 4ms/step - loss: 0.1847 - accuracy: 1.0000
Epoch 419/550
1/1 [==============================] - 0s 4ms/step - loss: 0.1824 - accuracy: 1.0000
Epoch 420/550
1/1 [==============================] - 0s 4ms/step - loss: 0.1799 - accuracy: 1.0000
Epoch 421/550
1/1 [==============================] - 0s 4ms/step - loss: 0.1774 - accuracy: 1.0000
Epoch 422/550
1/1 [==============================] - 0s 3ms/step - loss: 0.1751 - accuracy: 1.0000
Epoch 423/550
1/1 [=====

1/1 [==============================] - 0s 3ms/step - loss: 0.0709 - accuracy: 1.0000
Epoch 496/550
1/1 [==============================] - 0s 3ms/step - loss: 0.0701 - accuracy: 1.0000
Epoch 497/550
1/1 [==============================] - 0s 3ms/step - loss: 0.0694 - accuracy: 1.0000
Epoch 498/550
1/1 [==============================] - 0s 3ms/step - loss: 0.0686 - accuracy: 1.0000
Epoch 499/550
1/1 [==============================] - 0s 4ms/step - loss: 0.0679 - accuracy: 1.0000
Epoch 500/550
1/1 [==============================] - 0s 4ms/step - loss: 0.0672 - accuracy: 1.0000
Epoch 501/550
1/1 [==============================] - 0s 3ms/step - loss: 0.0665 - accuracy: 1.0000
Epoch 502/550
1/1 [==============================] - 0s 4ms/step - loss: 0.0658 - accuracy: 1.0000
Epoch 503/550
1/1 [==============================] - 0s 4ms/step - loss: 0.0651 - accuracy: 1.0000
Epoch 504/550
1/1 [==============================] - 0s 4ms/step - loss: 0.0644 - accuracy: 1.0000
Epoch 505/550
1/1 [=====

In [9]:
import pickle
# saving model
model.save("chat_model")




# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets
